# Echelon Form and Row Reduced Echelon Form of a Rectangular System with applications: Block Matrix Operation, Matrix Inversion, Spanning sets of all the four fundamental subspaces, Image compression

---
This notebook shows how to compute the Row Reduced Echelon Form using the Symbolic Package (Sympy). The the implementation of Echelon Form, Echelon Form with Partial Pivoting, Row Reduced Echelon Form and Row Reduced Echelon Form with Partial Pivoting, with related example.


## Import libraries

In [1]:
import sympy as sym
import numpy as np

##  Computing the Row Reduced Echelon Form using the method *rref* from **symbolic package** (***sympy***)

### Test on a matrix **A** with 3 rows and 7 columns

In [2]:
# Definition of the matrix A
A = sym.Matrix([[1, 2, 2, 3, 1, 0, 0],
                [2, 4, 1, 3, 0,1,0],
                [3, 6, 1, 4,0,0,1]])
print("Matrix A: {} ".format(A))

# Use sympy.rref() method to compute the row reduce echelon form.
# The method rref return a matrix in the row reduce echelon form and
# a vector with the index of pivots (position of basic columns in the matrix).
A_rref = A.rref()

# print the row reduce  form of the matrix A and the indexes of the basic columns
print("\nThe Row Reduced Echelon Form of matrix A and the pivot columns are: {}"
      .format(A_rref))

Matrix A: Matrix([[1, 2, 2, 3, 1, 0, 0], [2, 4, 1, 3, 0, 1, 0], [3, 6, 1, 4, 0, 0, 1]]) 

The Row Reduced Echelon Form of matrix A and the pivot columns are: (Matrix([
[1, 2, 0, 1, 0, -1,  1],
[0, 0, 1, 1, 0,  3, -2],
[0, 0, 0, 0, 1, -5,  3]]), (0, 2, 4))


### Test on a matrix **B** with 4 rows and 6 columns

In [3]:
# Definition of the matrix B
B = sym.Matrix([[1, 1, 2, 2, 1, 1], [2, 2, 4, 4, 3, 1],
                [2, 2, 4, 4, 2, 2], [3, 5, 8, 6, 5, 3]])
print("Matrix B: {} ".format(B))

B_rref = B.rref()

# Print the row reduce form of the matrix B and the indexes of the basic columns
print("\nThe Row Reduced Echelon Form of matrix B and the pivot columns are: {}"
      .format(B_rref))

Matrix B: Matrix([[1, 1, 2, 2, 1, 1], [2, 2, 4, 4, 3, 1], [2, 2, 4, 4, 2, 2], [3, 5, 8, 6, 5, 3]]) 

The Row Reduced Echelon Form of matrix B and the pivot columns are: (Matrix([
[1, 0, 1, 2, 0,  1],
[0, 1, 1, 0, 0,  1],
[0, 0, 0, 0, 1, -1],
[0, 0, 0, 0, 0,  0]]), (0, 1, 4))


## Echelon Form, Echelon Form with Partial Pivoting, Row Reduced Echelon Form and Row Reduced Echelon Form with Partial Pivoting

We define a function which takes in Input a Matrix of size m x n and that gives in output the row reduced echelon form echelon form of it and the related permutation matrix. The permutation is performed in order to have a stable algorithm using the technique of partial pivoting.

### Function to compute the Echelon Form
* INPUT
  * **A**: Input matrix A
* OUTPUT
  * **U**: The Echelon Form of the input matrix A
  * **ind_basic_cols**: The position of basic columns into the input matrix

In [4]:
def echelon_form(A):
  # In this implementation we are supposing that
  # the number of columns is greater or equal the number of rows

  # Retrive the dimension of the matrix A:
  # m: number of rows
  # n: number of columns
  (m, n) = A.shape

  # Copy the matrix A in U, which will stored the echelon matrix
  U = np.copy(A)

  # Vector to store the positions of pivotal elements (basic columns)
  ind_basic_cols = []

  # j: index related to the columns; j take into account
  # when we move from a pivot columns to a non-pivot columns
  j = 0

  # i: index related to the rows
  for i in range(0, m):
    # Uset to check if we need to move to the next column
    ech = True

    while (ech) & (j < n):
      # Find the pivotal index, maximum element in the column j, below the
      # main diagonal. In fact, we search the maximum element in absolute
      # value between the values in column j, but in row from i to m.
      # Basically, we are not consider the values in the pivotal position.
      indm = np.argmax(abs(U[i:m, j]))

      # Then we store the index of the maximum value in columns
      # (we add i because we neew to take into account the original position
      # into the original matrix)
      indm = indm + i

      # If the index of max pivot on column j is different from
      # the diagonal index i and the element in position idm is greater than
      # 1e-15 (which is something near to zero) we can perform the row exchange.
      #
      # 1) If abs(U[indm,j]) > 1e-15, this means that all the
      # elements on column j are > 1e-15.
      #
      # 2) We put > 1e-15 instead of != 0 because in the machine
      # is not correct check (=0 or != 0), so we check with a
      # very small value which is near to zero.
      if abs(U[i, j]) > 1e-15:
        # Perform permutation between row i and row indm
        # to move the greatest value in column in the pivotal position
        U[[i, indm], :] = U[[indm, i], :]

      # If the pivotal element is different from zero,
      # we can apply the rows elimination
      if abs(U[i, j]) > 1e-15:
        # Store the index of pivotal element (basic column)
        ind_basic_cols.append(j)

        # We compute in one time all the coefficients M used to make the
        # row elimination and we store in one vector.
        # In this way, we can compute the elimination of all the row
        # below the pivotal element in one time.
        M = U[i + 1 : m, j] / U[i, j]

        # In numpy array there is no difference row vectors or column vectors
        # np.outer to perform the outer product between two vectors.
        # We perform all the row linear combination with one statement
        # A(*,*) = A(*,*) - m A(*,*)
        U[i + 1 : m, j + 1 : n] = U[i + 1 : m, j + 1 : n] - np.outer(M, U[i, j + 1 : n])

        # We put 0 below the pivotal element
        U[i + 1 : m, j] = 0

        j = j + 1
        ech = False
      else:
        # The column is not a pivot column,
        # so we can move to find the next
        j = j + 1
        ech = True
  return (U, ind_basic_cols)

#### Test on a matrix **C** with 3 rows and 4 columns

In [5]:
C = np.array([[1., 2., 2., 3.], [2, 4, 1, 3], [3, 6, 1, 4]])
print("Matrix C: \n{}".format(C))

U, ind_basic_cols = echelon_form(C)
print("\nThe Echelon form of matrix C: \n{}".format(U))
print("\nThe basic columns of matrix C: {}".format(ind_basic_cols))

Matrix C: 
[[1. 2. 2. 3.]
 [2. 4. 1. 3.]
 [3. 6. 1. 4.]]

The Echelon form of matrix C: 
[[3.00000000e+00 6.00000000e+00 1.00000000e+00 4.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.66666667e+00 1.66666667e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.11022302e-16]]

The basic columns of matrix C: [0, 2]


### Function to compute the Echelon Form with Partial Pivoting
* INPUT
  * **A**: Input matrix A
* OUTPUT
  * **U**: The Echelon Form using Partial Pivoting of the input matrix A
  * **ind_basic_cols**: The position of basic columns into the input matrix
  * **perm_matrix**: The permutation matrix used to compute the Partial Pivoting

In [6]:
def echelon_form_with_partial_pivot(A):
  # In this implementation we are supposing that
  # the number of columns is greater or equal the number of rows

  # Retrive the dimension of the matrix A:
  # m: number of rows
  # n: number of columns
  (m, n) = A.shape

  # Copy the matrix A in U, which will stored the echelon matrix
  U = np.copy(A)

  # Create an identity matrix of size mxm
  # to use as permutation matrix
  perm_matrix = np.eye(m)

  # Vector to store the positions of pivotal elements (basic columns)
  ind_basic_cols = []

  # j: index related to the columns; j take into account
  # when we move from a pivot columns to a non-pivot columns
  j = 0

  # i: index related to the rows
  for i in range(0, m):
    # Used to check if we need to move to the next column
    ech = True

    while (ech) & (j < n):
      # Find the pivotal index, maximum element in the column j, below the
      # main diagonal. In fact, we search the maximum element in absolute
      # value between the values in column j, but in row from i to m.
      # Basically, we are not consider the values in the pivotal position.
      indm = np.argmax(abs(U[i:m, j]))

      # Then we store the index of the maximum value in columns
      # (we add i because we neew to take into account the original position
      # into the original matrix)
      indm = indm + i

      # If the index of max pivot on column j is different from
      # the diagonal index i and the element in position idm is greater than
      # 1e-15 (which is something near to zero) we can perform the row exchange.

      # 1) If abs(U[indm,j]) > 1e-15, this means that all the
      # elements on column j are > 1e-15.

      # 2) We put > 1e-15 instead of != 0 because in the machine
      # is not correct check (=0 or != 0), so we check with a
      # very small value which is near to zero.
      if (indm != i) & (abs(U[indm, j]) > 1e-15):
        # Perform permutation between row i and row indm
        # to move the greatest value in column in the pivotal position
        U[[i, indm], :] = U[[indm, i], :]

        # Update the permutation matrix to save all the
        # permutation that we will perform
        perm_matrix[[i, indm], :] = perm_matrix[[indm, i], :]

      # If the pivotal element is different from zero,
      # we can apply the rows elimination
      if abs(U[i, j]) > 1e-15:
        # Store the index of pivotal element (basic column)
        ind_basic_cols.append(j)

        # We compute in one time all the coefficients M used to make the
        # row elimination and we store in one vector.
        # In this way, we can compute the elimination of all the row
        # below the pivotal element in one time.
        M = U[i + 1 : m, j] / U[i, j]

        # In numpy array there is no difference row vectors or column vectors
        # np.outer to perform the outer product between two vectors.
        # We perform all the row linear combination with one statement
        # A(*,*) = A(*,*) - m A(*,*)
        U[i + 1 : m, j + 1 : n] = U[i + 1 : m, j + 1 : n] - np.outer(M, U[i, j + 1 : n])

        # We put 0 below the pivotal element
        U[i + 1 : m, j] = 0

        j = j + 1
        ech = False
      else:
        # The column is not a pivot column,
        # so we can move to find the next
        j = j + 1
        ech = True
  return (U, ind_basic_cols, perm_matrix)

#### Test on a matrix **C** with 3 rows and 4 columns

In [7]:
C = np.array([[1., 2., 2., 3.], [2., 4., 1., 3.], [3., 6., 1., 4.]])
print("Matrix C: \n{}".format(C))

U, ind_basic_cols, perm_matrix = echelon_form_with_partial_pivot(C)
print("\nThe Echelon Form with Partial Pivoting of matrix C: \n{}".format(U))
print("\nThe basic columns of matrix C: {}".format(ind_basic_cols))
print("\nThe Permutation matrix used to perform the Partial Pivoting on C:\n{}\n".format(perm_matrix))
print("Computing by hands the Echelon Form with Partial Pivoting of C, we will find that rank(C) = 2.\n" +
      "Looking at U, from a theoretically point of view, we should consider that rank(C) = 3 because 1.11022302e-16 =/= 0.\n" +
      "From a numerically point of view 1.11022302e-16 is something very small (roundoff error).\n" +
      "So, we can consider zero, which means that rank(C) = 2.")

Matrix C: 
[[1. 2. 2. 3.]
 [2. 4. 1. 3.]
 [3. 6. 1. 4.]]

The Echelon Form with Partial Pivoting of matrix C: 
[[3.00000000e+00 6.00000000e+00 1.00000000e+00 4.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.66666667e+00 1.66666667e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.11022302e-16]]

The basic columns of matrix C: [0, 2]

The Permutation matrix used to perform the Partial Pivoting on C:
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]

Computing by hands the Echelon Form with Partial Pivoting of C, we will find that rank(C) = 2.
Looking at U, from a theoretically point of view, we should consider that rank(C) = 3 because 1.11022302e-16 =/= 0.
From a numerically point of view 1.11022302e-16 is something very small (roundoff error).
So, we can consider zero, which means that rank(C) = 2.


### Function to compute the Row Reduced Echelon Form
* INPUT
  * **A**: Input matrix A
* OUTPUT
  * **U**: The Row Reduced Echelon Form of the input matrix A
  * **ind_basic_cols**: The position of basic columns into the input matrix A

In [8]:
def row_reduced_echelon_form(A):
  # In this implementation we are supposing that
  # the number of columns is greater or equal the number of rows

  # Retrive the dimension of the matrix A:
  # m: number of rows
  # n: number of columns
  (m, n) = A.shape

  # Copy the matrix A in U, which will stored the echelon matrix
  U = np.copy(A)

  # Vector to store the positions of pivotal elements (basic columns)
  ind_basic_cols = []

  # j: index related to the columns; j take into account
  # when we move from a pivot columns to a non-pivot columns
  j = 0

  # i: index related to the rows
  for i in range(0,m):
    # Used to check if we need to move to the next column
    ech = True

    while (ech) & (j < n):
      # Find the pivotal index, maximum element in the column j, below the
      # main diagonal. In fact, we search the maximum element in absolute
      # value between the values in column j, but in row from i to m.
      # Basically, we are not consider the values in the pivotal position.
      indm = np.argmax(abs(U[i:m, j]))

      # Then we store the index of the maximum value in columns
      # (we add i because we neew to take into account the original position
      # into the original matrix)
      indm = indm + i

      # # If the index of max pivot on column j is different from
      # the diagonal index i and the element in position idm is greater than
      # 1e-15 (which is something near to zero) we can perform the row exchange.
      #
      # 1) If abs(U[indm,j]) > 1e-15, this means that all the
      # elements on column j are > 1e-15.
      #
      # 2) We put > 1e-15 instead of != 0 because in the machine
      # is not correct check (=0 or != 0), so we check with a
      # very small value which is near to zero.
      if  (abs(U[indm, j]) > 1e-15):
        # Perform permutation between row i and row idm
        U[ [i, indm], :] = U[[indm, i], :]

      # If the pivotal element is different from zero,
      # we can apply the rows elimination
      if ( abs(U[i,j]) > 1e-15):
        # Store the index of pivotal element (basic column)
        ind_basic_cols.append(j)

        # We compute in one time all the coefficients M used to make the
        # row elimination and we store in one vector.
        # In this way, we can compute the elimination of all the row
        # below the pivotal element in one time.
        M = U[i + 1: m, j] / U[i, j]

        # In numpy array there is no difference row vectors or column vectors
        # np.outer to perform the outer product between two vectors.
        # We perform all the row linear combination with one statement
        # A(*,*) = A(*,*) - m A(*,*)
        U[i + 1 : m, j + 1 : n] = U[i + 1 : m, j + 1 : n] - np.outer(M, U[i, j + 1 : n])

        # We put 0 below the pivotal element
        U[i+1:m,j]=0

        # Divided all the row by the pivotal element in order to put the pivotal
        # element equals to 1
        U[i, j : n] = U[i, j : n] / U[i, j]

        if i > 0:
          # If we are working on a row different from the first row, we need to
          # perform linear combination in order to put zero also above the pivot.
          # So, we compute the coefficent M needed to comput the row elimination.
          M = U[0: i, j]/U[i, j]
          U[0 : i, j : n] = U[0: i, j: n] - np.outer(M, U[i, j : n])

        j = j + 1
        ech = False
      else:
        # The column is not a pivot column,
        # so we can move to find the next
        j = j + 1
        ech= True
  return(U, ind_basic_cols)

#### Test on a matrix **D** with 3 rows and 7 columns

In [9]:
D = np.array([[1., 2., 2., 3., 1., 0., 0.], [2., 4., 1., 3., 0., 1., 0.], [3., 6., 1., 4., 0., 0., 1.]])
print("Matrix D: \n{}".format(D))

U, ind_basic_cols = row_reduced_echelon_form(D)
print("\nThe Row Reduced Echelon form of matrix D: \n{}".format(U))
print("\nThe basic columns of matrix D: \n{}".format(ind_basic_cols))

Matrix D: 
[[1. 2. 2. 3. 1. 0. 0.]
 [2. 4. 1. 3. 0. 1. 0.]
 [3. 6. 1. 4. 0. 0. 1.]]

The Row Reduced Echelon form of matrix D: 
[[ 1.00000000e+00  2.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00 -1.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  1.00000000e+00
   0.00000000e+00  3.00000000e+00 -2.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.11022302e-16
   1.00000000e+00 -5.00000000e+00  3.00000000e+00]]

The basic columns of matrix D: 
[0, 2, 4]


### Function to compute the Row Reduced Echelon Form with Partial Pivoting
* INPUT
  * **A**: Input matrix A
* OUTPUT
  * **U**: The Row Reduced Echelon Form using Partial Pivoting of the input matrix A
  * **ind_basic_cols**: The position of basic columns into the input matrix A
  * **perm_matrix**: The permutation matrix used to compute the Partial Pivoting

In [10]:
def row_reduced_echelon_form_with_partial_pivot(A):
  # In this implementation we are supposing that
  # the number of columns is greater or equal the number of rows

  # Retrive the dimension of the matrix A:
  # m: number of rows
  # n: number of columns
  (m, n) = A.shape

  # Copy the matrix A in U, which will stored the echelon matrix
  U = np.copy(A)

  # Create an identity matrix of size mxm
  # to use as permutation matrix
  perm_matrix = np.eye(m)

  # Vector to store the positions of pivotal elements (basic columns)
  ind_basic_cols = []

  # j: index related to the columns; j take into account
  # when we move from a pivot columns to a non-pivot columns
  j = 0

  # i: index related to the rows
  for i in range(0,m):
    # Used to check if we need to move to the next column
    ech = True

    while (ech) & (j < n):
      # Find the pivotal index, maximum element in the column j, below the
      # main diagonal. In fact, we search the maximum element in absolute
      # value between the values in column j, but in row from i to m.
      # Basically, we are not consider the values in the pivotal position.
      indm = np.argmax(abs(U[i:m, j]))

      # Then we store the index of the maximum value in columns
      # (we add i because we neew to take into account the original position
      # into the original matrix)
      indm = indm + i

      # # If the index of max pivot on column j is different from
      # the diagonal index i and the element in position idm is greater than
      # 1e-15 (which is something near to zero) we can perform the row exchange.
      #
      # 1) If abs(U[indm,j]) > 1e-15, this means that all the
      # elements on column j are > 1e-15.
      #
      # 2) We put > 1e-15 instead of != 0 because in the machine
      # is not correct check (=0 or != 0), so we check with a
      # very small value which is near to zero.
      if (indm != i) & (abs(U[indm,j]) > 1e-15):

        # Perform permutation between row i and row idm
        U[[i, indm], :] = U[[indm, i], :]

        # Update the permutation matrix to save all the
        # permutation that we will perform
        perm_matrix[ [i, indm],:] = perm_matrix[[indm,i],:]

      # If the pivotal element is different from zero,
      # we can apply the rows elimination
      if (abs(U[i,j]) > 1e-15):

        # Store the index of pivotal element (basic column)
        ind_basic_cols.append(j)

        # We compute in one time all the coefficients M used to make the
        # row elimination and we store in one vector.
        # In this way, we can compute the elimination of all the row
        # below the pivotal element in one time.
        M = U[i + 1 : m, j] / U[i, j]

        # In numpy array there is no difference row vectors or column vectors
        # np.outer to perform the outer product between two vectors.
        # We perform all the row linear combination with one statement
        # A(*,*) = A(*,*) - m A(*,*)
        U[i + 1 : m, j + 1 : n] = U[i+1:m,j+1:n] - np.outer(M, U[i, j + 1 : n])

        # We put 0 below the pivotal element
        U[i + 1 : m, j] = 0

        # Divided all the row by the pivotal element in order to put the pivotal
        # element equals to 1
        U[i, j : n] = U[i, j : n] / U[i, j]

        if i > 0:
          # If we are working on a row different from the first row, we need to
          # perform linear combination in order to put zero also above the pivot.
          # So, we compute the coefficent M needed to comput the row elimination.
          M= U[0 : i , j] / U[i, j]
          U[0 : i, j : n] = U[0 : i, j : n] - np.outer(M, U[i, j : n])

        j = j + 1
        ech = False
      else:
        # The column is not a pivot column,
        # so we can move to find the next
        j = j + 1
        ech = True

  return(U, ind_basic_cols, perm_matrix)

#### Test on a matrix **D** with 3 rows and 7 columns

In [11]:
D = np.array([[1., 2., 2., 3., 1., 0., 0.], [2., 4., 1., 3., 0., 1., 0.], [3., 6., 1., 4., 0., 0., 1.]])
print("Matrix D: \n{}".format(D))

U, ind_basic_cols, perm_matrix = row_reduced_echelon_form_with_partial_pivot(D)
print("\nThe Row Reduced Echelon form of matrix D: \n{}".format(U))
print("\nThe basic columns of matrix D: {}".format(ind_basic_cols))
print("\nThe Permutation matrix used to perform the Partial Pivoting on D: \n{}".format(perm_matrix))

Matrix D: 
[[1. 2. 2. 3. 1. 0. 0.]
 [2. 4. 1. 3. 0. 1. 0.]
 [3. 6. 1. 4. 0. 0. 1.]]

The Row Reduced Echelon form of matrix D: 
[[ 1.00000000e+00  2.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00 -1.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  1.00000000e+00
   0.00000000e+00  3.00000000e+00 -2.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.11022302e-16
   1.00000000e+00 -5.00000000e+00  3.00000000e+00]]

The basic columns of matrix D: [0, 2, 4]

The Permutation matrix used to perform the Partial Pivoting on D: 
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]


## Applications of Row Reduced Echelon Form